In [2]:
#---import libraries relating to data manipulation, visualization---------------------------------------------------
import pandas as pd
%matplotlib inline
import numpy as np
import json


#-------import libraries to evaluate the deep learning model---------------------------------------------------------------
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore')

#------ import Deep Learning Libraries-------#

import torch
import torch.nn as nn
import torch.optim as optim

import optuna
from optuna.trial import Trial

import pickle


pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

import sys
sys.path.append('../../../src')  # Add the 'src' directory to the path

#### Import Data Loading and Processing Functions
from multimodal.helper_functions import   reproducible_comp,  get_final_df
from multimodal.model_architecture import Generic1DCNN
reproducible_comp()

In [4]:
file_path = "../../../data/processed_data/acc_inner.pkl"
with open(file_path, 'rb') as picklefile:
    data = pickle.load(picklefile)

features_acc_inner, category_acc_inner, category_names, df_acc_inner = get_final_df(data, is_acc_data=1, is_multilabel=1)

labels = category_acc_inner.copy()
features=features_acc_inner.copy()
print("Features Shape: ", features.shape)
print("Labels Shape: ", labels.shape)

Features Shape:  (1350, 3)
Labels Shape:  (1350, 4)


In [6]:
max_length = 500

for i in range (len(features)):
    if(len(features[i][0]<max_length)):
        features[i][0]=np.pad(features[i][0], (0,max_length-len(features[i][0])), 'constant', constant_values=(0))
        
    if(len(features[i][1]<max_length)):
        features[i][1]=np.pad(features[i][1], (0,max_length-len(features[i][1])), 'constant', constant_values=(0))
    
    if(len(features[i][2]<max_length)):
        features[i][2]=np.pad(features[i][2], (0,max_length-len(features[i][2])), 'constant', constant_values=(0))

In [7]:
feat= np.zeros(len(features)*3*max_length).reshape(-1, 3*max_length)
for i in range(len(features)):
    feat[i]= np.concatenate((features[i][0],features[i][1], features[i][2]), axis=None)
    

print("Updated Features Shape: ", feat.shape)

Updated Features Shape:  (1350, 1500)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(feat, labels, test_size=0.1, 
                                                    random_state=2, shuffle=True, stratify=labels)

In [9]:
X_train = X_train.reshape(-1, 1, feat.shape[1])
X_test = X_test.reshape(-1, 1, feat.shape[1])

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1215, 1, 1500)
(1215, 4)
(135, 1, 1500)
(135, 4)


In [10]:
from torch.utils.data import DataLoader, TensorDataset

# Convert your data to PyTorch tensors
x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

input_channels = 1
num_classes = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the objective function for hyperparameter optimization
def objective(trial: Trial):
    try:
        # Define hyperparameters to search for
        lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)  # Suggest a learning rate in log scale
        dropout_prob = trial.suggest_float("dropout_prob", 0.0, 0.5)
        hidden_channels = trial.suggest_int("hidden_channels", 16, 256)
        n_layers = trial.suggest_int("n_layers", 1, 7)
        kernel_size = trial.suggest_int("kernel_size", 3, 9)
        stride = trial.suggest_int("stride", 1, 7)
        padding = trial.suggest_int("padding", 1, 10)
        pooling_type = trial.suggest_categorical("pooling_type", ["max", "avg"])  
        use_batch_norm = trial.suggest_categorical("use_batch_norm", [True, False]) 


        # Split the data into training and validation sets
        x_train_split, x_valid_split, y_train_split, y_valid_split = train_test_split(
            x_train, y_train, test_size=0.2, random_state=42
        )

        train_dataset = TensorDataset(x_train_split, y_train_split)
        valid_dataset = TensorDataset(x_valid_split, y_valid_split)

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

        # Create the model with the suggested hyperparameters
        model = Generic1DCNN(input_channels, num_classes, n_layers, kernel_size, stride, padding,
                            hidden_channels, dropout_prob, pooling_type, use_batch_norm).to(device)

        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        num_epochs = 50

        model.train()
        for epoch in range(num_epochs):
            # Training loop
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                outputs = model(inputs)
                loss = criterion(outputs, labels.float())
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            train_loss = running_loss / len(train_loader)

            # Validation loss calculation (similar to training loop)
            model.eval()
            valid_loss = 0.0
            with torch.no_grad():
                for inputs_valid, labels_valid in valid_loader:
                    inputs_valid, labels_valid = inputs_valid.to(device), labels_valid.to(device)
                    outputs_valid = model(inputs_valid)
                    loss_valid = criterion(outputs_valid, labels_valid.float())
                    valid_loss += loss_valid.item()

            valid_loss = valid_loss / len(valid_loader)
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss}, Validation Loss: {valid_loss}")

        # Calculate and return the final validation loss
        return valid_loss
    
    except Exception as e:
        # Print the error for debugging purposes (optional)
        print(f"Trial failed with error: {e}")

        # Return a high validation loss to indicate failure
        return 1.0  # High validation loss
    


In [11]:
# Create an Optuna study with SQLite storage and TPE pruner
study = optuna.create_study(
    study_name="Case#4: Acceleration Inner",
    direction="minimize",  # Minimize final validation loss
    storage="sqlite:///../../database_sqllite//db.sqlite3",
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),  
)

# Define the number of trials (hyperparameter combinations) you want to search
num_trials = 100  

# Perform hyperparameter optimization
study.optimize(objective, n_trials=num_trials, n_jobs=1)


[I 2023-10-09 17:48:49,674] A new study created in RDB with name: Case#4: Acceleration Inner


Epoch 10/50, Training Loss: 0.3968174681067467, Validation Loss: 0.4001191556453705
Epoch 20/50, Training Loss: 0.31388004682958126, Validation Loss: 0.3330627456307411
Epoch 30/50, Training Loss: 0.20432480797171593, Validation Loss: 0.3121752440929413
Epoch 40/50, Training Loss: 0.17888879496604204, Validation Loss: 0.3009425327181816


[I 2023-10-09 17:49:42,006] Trial 0 finished with value: 0.36313892155885696 and parameters: {'lr': 0.0007978607177258467, 'dropout_prob': 0.48463239206933706, 'hidden_channels': 183, 'n_layers': 5, 'kernel_size': 5, 'stride': 3, 'padding': 10, 'pooling_type': 'max', 'use_batch_norm': False}. Best is trial 0 with value: 0.36313892155885696.


Epoch 50/50, Training Loss: 0.12375963758677244, Validation Loss: 0.36313892155885696
Epoch 10/50, Training Loss: 0.579501960426569, Validation Loss: 0.5861624181270599
Epoch 20/50, Training Loss: 0.4677633661776781, Validation Loss: 0.4728834629058838
Epoch 30/50, Training Loss: 0.3753711674362421, Validation Loss: 0.40757355093955994
Epoch 40/50, Training Loss: 0.31639768928289413, Validation Loss: 0.37517624348402023


[I 2023-10-09 17:50:04,697] Trial 1 finished with value: 0.3542170152068138 and parameters: {'lr': 0.0003711651291173978, 'dropout_prob': 0.4132796088773777, 'hidden_channels': 223, 'n_layers': 2, 'kernel_size': 6, 'stride': 7, 'padding': 9, 'pooling_type': 'avg', 'use_batch_norm': True}. Best is trial 1 with value: 0.3542170152068138.


...

Epoch 50/50, Training Loss: 0.23440580163151026, Validation Loss: 0.2974071279168129
Epoch 10/50, Training Loss: 0.3715367019176483, Validation Loss: 0.3870939686894417
Epoch 20/50, Training Loss: 0.2523622317239642, Validation Loss: 0.3115697279572487
Epoch 30/50, Training Loss: 0.17719637602567673, Validation Loss: 0.2849443554878235
Epoch 40/50, Training Loss: 0.13867640169337392, Validation Loss: 0.28732798993587494


[I 2023-10-09 19:32:16,514] Trial 98 finished with value: 0.3419702649116516 and parameters: {'lr': 0.0001328100483506015, 'dropout_prob': 0.3366969794931325, 'hidden_channels': 218, 'n_layers': 5, 'kernel_size': 9, 'stride': 1, 'padding': 5, 'pooling_type': 'avg', 'use_batch_norm': True}. Best is trial 82 with value: 0.2734515219926834.


Epoch 50/50, Training Loss: 0.09788085939362645, Validation Loss: 0.3419702649116516
Epoch 10/50, Training Loss: 0.42618376947939396, Validation Loss: 0.41551174223423004
Epoch 20/50, Training Loss: 0.2972598932683468, Validation Loss: 0.3645159900188446
Epoch 30/50, Training Loss: 0.2164282128214836, Validation Loss: 0.33888906985521317
Epoch 40/50, Training Loss: 0.17236564308404922, Validation Loss: 0.32960981875658035


[I 2023-10-09 19:36:45,481] Trial 99 finished with value: 0.36982356011867523 and parameters: {'lr': 9.759823383670421e-05, 'dropout_prob': 0.34992630732538843, 'hidden_channels': 255, 'n_layers': 5, 'kernel_size': 8, 'stride': 1, 'padding': 6, 'pooling_type': 'avg', 'use_batch_norm': True}. Best is trial 82 with value: 0.2734515219926834.


Epoch 50/50, Training Loss: 0.15143727138638496, Validation Loss: 0.36982356011867523


In [12]:
# Print the best hyperparameters and their associated final validation loss
print("Best hyperparameters:")
print(study.best_params)
print("Best final validation loss:")
print(study.best_value)

Best hyperparameters:
{'lr': 0.00018137907646630452, 'dropout_prob': 0.3524356687822107, 'hidden_channels': 219, 'n_layers': 4, 'kernel_size': 9, 'stride': 2, 'padding': 6, 'pooling_type': 'avg', 'use_batch_norm': True}
Best final validation loss:
0.2734515219926834


In [13]:
# Specify the path where you want to save the JSON file
config_file_path = "../best_configs/acc_inner_best_configs.json"

In [15]:
# Load the saved configuration
with open(config_file_path, "r") as config_file:
    saved_configs = json.load(config_file)

# Compare the new best value with the saved one
if study.best_value < saved_configs["best_value"]:
    print("New best value found. Replacing saved configuration.")
    saved_configs["best_params"] = study.best_params
    saved_configs["best_value"] = study.best_value

    # Update the saved configuration file
    with open(config_file_path, "w") as config_file:
        json.dump(saved_configs, config_file)

In [14]:
# Save the best hyperparameters and validation loss to a JSON file
best_configs = {
    "best_params": study.best_params,
    "best_value": study.best_value,
}

with open(config_file_path, "w") as config_file:
    json.dump(best_configs, config_file)

# Thank You